In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langgraph.graph import StateGraph, END 
from langchain.chat_models import ChatOpenAI
from typing import Dict, TypedDict, List, Annotated, Optional, Any
import operator, sqlite3
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, FunctionMessage
import json 
import re 
from enum import Enum
from pydantic import BaseModel, Field
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [3]:
llm = ChatOpenAI(temperature=0.7, model_name= "gpt-4o")

C:\Users\Sahruz Riyad\AppData\Local\Temp\ipykernel_8788\4016619961.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, model_name= "gpt-4o")


In [4]:
class User(BaseModel):
    user_id: Optional[int] = None
    name: str
    age: int 
    email: str 
    order_list: Optional[List[Dict]] = None
    interested_in: Optional[List[str]] = None

In [5]:
class EcommerceState(TypedDict):
    user_info: Dict
    user_input: str

Database Set-up

In [6]:
def connect_db():
    conn = sqlite3.connect("user.db")
    return conn 

def create_table(cursor):
    """Creates the users table if it doesn't exist."""
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            age INTEGER,
            email TEXT UNIQUE,
            order_list TEXT,
            interested_in TEXT
        )
    ''')

def save_user(cursor, user: User):
    """Saves a User object to the database."""
    order_list_json = json.dumps(user.order_list) if user.order_list else None
    interested_in_json = json.dumps(user.interested_in) if user.interested_in else None

    if user.user_id is None: #insert new user
        cursor.execute('''
            INSERT INTO users (name, age, email, order_list, interested_in)
            VALUES (?, ?, ?, ?, ?)
        ''', (user.name, user.age, user.email, order_list_json, interested_in_json))
        user.user_id = cursor.lastrowid #get the generated id
    else: #update existing user
         cursor.execute('''
            UPDATE users SET name = ?, age = ?, email = ?, order_list = ?, interested_in = ?
            WHERE user_id = ?
        ''', (user.name, user.age, user.email, order_list_json, interested_in_json, user.user_id))

def add_users_sample():
    conn = connect_db()
    cursor = conn.cursor()

    create_table(cursor)

    user1 = User(name="Alice", age=30, email="alice@example.com", order_list=[{"item": "book", "quantity": 1}], interested_in=["reading", "coding"])
    user2 = User(name="Bob", age=25, email="bob@example.com", interested_in=["sports"])
    user3 = User(name="Charlie", age=35, email="charlie@example.com", order_list=[], interested_in=[])

    save_user(cursor, user1)
    save_user(cursor, user2)
    save_user(cursor, user3)

    conn.commit()
    conn.close()

In [7]:
# add_users_sample()

In [8]:
def fetch_user_info(email):
    conn = connect_db()
    cursor = conn.cursor()

    cursor.execute("SELECT name, email, order_list, interested_in FROM users WHERE email=?", (email,))
    
    user = cursor.fetchone()
    conn.close()
    
    if user:
        return {
            "name": user[0],
            "email": user[1],
            "order_list": user[2],
            "interested_in": user[3]
        }
    else:
        return {
            "name": "Guest",
            "email": "",
            "order_list": "",
            "interested_in": ""
        }

In [9]:
print(json.dumps(fetch_user_info("alice@example.com")))

{"name": "Alice", "email": "alice@example.com", "order_list": "[{\"item\": \"book\", \"quantity\": 1}]", "interested_in": "[\"reading\", \"coding\"]"}


In [10]:
user_info = fetch_user_info("alice@example.com")
print(user_info)

{'name': 'Alice', 'email': 'alice@example.com', 'order_list': '[{"item": "book", "quantity": 1}]', 'interested_in': '["reading", "coding"]'}


In [11]:
def greet_user(state: EcommerceState):
    print(state['user_info'])
    return {"message": f"Hello {state['user_info']['name']}! Welcome to our store. How can I assist you today?"}

In [27]:
def get_user_intent(state: EcommerceState):
    user_input = state[1].lower()

    if "browse" in user_input:
        return "browse"
    elif "order" in user_input:
        return "order_status"
    elif "support" in user_input:
        return "support"
    elif "recommend" in user_input:
        return "recommendations"
    else:
        return "unknown"

In [13]:
def browse_product(state: EcommerceState):
    return {"message": "Here are some popular categories: Electronics, Fashion, Home & Kitchen. What interests you?"}


In [14]:
def check_order_status(state: EcommerceState):
    return {"message": "Please provide your order ID to check the status."}

In [15]:
def customer_support(state: EcommerceState):
    return {"message": "What issue are you facing? I can assist with refunds, tracking, or general inquiries."}


In [16]:
def personalized_recommendations(state: EcommerceState):
    return {"message": f"Based on your past orders: {state['user_info']['interested_in']}, you might like these items: [Product A], [Product B], [Product C]."}


In [17]:
def fallback(state: EcommerceState):
    return {"message": "I'm not sure how to assist with that. Can you please rephrase?"}

In [18]:
from langgraph.graph import Graph
graph = Graph()

In [19]:
graph.add_node("greet_user",greet_user)
graph.add_node("intent", get_user_intent)
graph.add_node("browse", browse_product)
graph.add_node("order_status", check_order_status)
graph.add_node("support", customer_support)
graph.add_node("recommendations", personalized_recommendations)
graph.add_node("fallback", fallback)

In [20]:
graph.add_edge("greet_user", "intent")

In [21]:
# Conditional routing
def route_intent(state: EcommerceState):
    intent = get_user_intent(state)
    return {
        "browse": "browse",
        "order_status": "order_status",
        "support": "support",
        "recommendations": "recommendations"
    }.get(intent, "fallback")

graph.add_conditional_edges(
    "intent",
    route_intent,
    {
        "browse": "browse",
        "order_status": "order_status",
        "support": "support",
        "recommendations": "recommendations",
        "fallback": "fallback"
    }
)

In [22]:
graph.set_entry_point("intent")

In [23]:
user_input = "I want recommendations"

In [24]:
workflow = graph.compile()

In [25]:
initial_state = EcommerceState(
    user_info=user_info,
    user_input=user_input
)

In [28]:
result = workflow.invoke(initial_state)

In [29]:
print(result)

None
